[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pronobis/libspn-keras/blob/master/examples/notebooks/Benchmark%20With%20Einsum%20Networks.ipynb)
# Bench-marking LibSPN-Keras against Einsum Networks
Recently, [Peharz et al.](https://arxiv.org/abs/2004.06231) proposed _Einsum Networks_, which are essentially SPNs with a specific implementation style. Einsum Networks implement a range of tricks that can be used to easily achieve tensor-based building of SPNs as well as EM learning through _autograd_. 

Fortunately, `libspn-keras` already had the same tricks implemented _ever since it was made available_ (before the Einsum paper was published), and so it was straightforward to set up a side-by-side comparison of `libspn-keras` 
against the open-source implementation of Einsum Networks.

## Installing Frameworks
`libspn-keras` is available on PyPI and so we'll simply use `pip` to install it. For `Einsum` we'll need to clone its repo and add it to our `PATH`.



In [ ]:
!git clone https://github.com/cambridge-mlg/EinsumNetworks & pip install libspn-keras --ignore-installed --no-deps



## Bench-marking Architecture
This notebook takes an architecture that was made available in the original Einsum Network repo. We increase the depth of the network and the number of sums and input distributions to get a better idea of how the `libspn-keras` and `Einsum` implementations scale.

The architecture is an SPN that recursively splits variables in two balanced groups starting at the root node. The groups are assigned randomly. Splitting is done up until a `depth` (in this case 5). After that, the remaining groups are combined through 'factorized input distributions'. This means that the binary leaf indicators are each weighted and the multiplied with the remaining nodes in a group. This is repeated `num_repetitions` times.


In [ ]:
depth = 5
num_repetitions = 32
num_input_distributions = 32
num_sums = 32

max_num_epochs = 3
batch_size = 100
online_em_frequency = 1
online_em_stepsize = 0.05

## Running Einsum
The public Einsum implementation uses a Pytorch backend. We'll run it for 10 epochs on a binary dataset and report
- The number of parameters per layer
- The log likelihood on validation data once every epoch
- The log likelihood on test data at the very end
- The total time spent training and computing the log-likelihood for the validation at each epoch

**NOTE**

For the most interesting comparison, make sure the GPU is actually being used. You can select a GPU in Colab using the `Runtime` dropdown which should show a `Change runtime type` option.

In [ ]:
import sys
sys.path.append("EinsumNetworks/src")
import time

import torch
from EinsumNetwork import Graph, EinsumNetwork
import datasets
import csv
import numpy as np

np.random.seed(1234)
torch.random.manual_seed(1234)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Running on ", device)

dataset = 'accidents'

train_x_orig, test_x_orig, valid_x_orig = datasets.load_debd(dataset, dtype='float32')

train_x = train_x_orig
test_x = test_x_orig
valid_x = valid_x_orig

# to torch
train_x = torch.from_numpy(train_x).to(torch.device(device))
valid_x = torch.from_numpy(valid_x).to(torch.device(device))
test_x = torch.from_numpy(test_x).to(torch.device(device))

train_N, num_dims = train_x.shape
valid_N = valid_x.shape[0]
test_N = test_x.shape[0]

graph = Graph.random_binary_trees(num_var=train_x.shape[1], depth=depth, num_repetitions=num_repetitions)

args = EinsumNetwork.Args(
    num_classes=1,
    num_input_distributions=num_input_distributions,
    exponential_family=EinsumNetwork.CategoricalArray,
    exponential_family_args={'K': 2},
    num_sums=num_sums,
    num_var=train_x.shape[1],
    online_em_frequency=1,
    online_em_stepsize=0.05)

einet = EinsumNetwork.EinsumNetwork(graph, args)
einet.initialize()
einet.to(device)
print(einet)

print("Number of input dist parameters")
input_layer_param_count = np.prod(einet.einet_layers[0].ef_array.params.shape)
print(input_layer_param_count)

print("Number of remaining sum parameters")
remaining_param_counts = [np.prod(layer.params.shape) for layer in einet.einet_layers if hasattr(layer,'params')]
print(remaining_param_counts)

print("Total trainable parameters")
print(sum([input_layer_param_count] + remaining_param_counts))

start = time.time()
for epoch_count in range(max_num_epochs):

    # evaluate
    with torch.no_grad():
        # This bit differs from original implementation 
        # (which did not disable grad, in other words this is to make this
        # part of the script run slightly faster)
        valid_ll = EinsumNetwork.eval_loglikelihood_batched(einet, valid_x)

    # Also here we only report the validation LLH during our training epochs
    # which makes it similar to how one would do it using Keras (or libspn-keras)
    print("[{}]   valid LL {}".format(epoch_count, valid_ll / valid_N))
    # train
    idx_batches = torch.randperm(train_N).split(batch_size)
    for batch_count, idx in enumerate(idx_batches):
        batch_x = train_x[idx, :]
        outputs = einet.forward(batch_x)

        ll_sample = EinsumNetwork.log_likelihoods(outputs)
        log_likelihood = ll_sample.sum()

        objective = log_likelihood
        objective.backward()

        einet.em_process_batch()

    einet.em_update()

print(f"Total time: {time.time() - start:.2f} seconds")
with torch.no_grad():
    test_ll = EinsumNetwork.eval_loglikelihood_batched(einet, test_x)
print(test_ll / test_N)


## Quick Analysis
We see the following interesting things:
- A test LLH of around -37.39
- A total train time of 148.64 seconds (depends of course on the hardware that this notebook runs on)
- Total number of trainable parameters 31,717,408 

## LibSPN-Keras Implementation
Instead of hiding all the complexity behind a `Graph.random_binary_trees`, `libspn-keras` offers a layer-based approach that gives you control over all network parameters flexibly at every layer. In that sense, we are in no sense forced to use the same number of sums for every layer, nor a fixed number of 2 children per product. We can easily vary all of these parameters. Nevertheless, for the sake of comparison we'll reimplement the above architecture using `libspn-keras`.

Let's see how they compare!

In [ ]:
import libspn_keras as spnk
import itertools
import tensorflow as tf

tf.random.set_seed(1234)

def build_ratspn(num_vars, depth):
    # Set global settings
    spnk.set_default_sum_op(spnk.SumOpEMBackprop())
    spnk.set_default_accumulator_initializer(tf.keras.initializers.RandomUniform())
    
    # Define stack
    sum_product_stack = [
        # Create repetitions, nodes and scope axes
        spnk.layers.FlatToRegions(num_decomps=num_repetitions, input_shape=[num_vars], dtype=tf.int32),
        # Add binary variables
        spnk.layers.IndicatorLeaf(num_components=2), 
        # Add dense sum layer (effectively computing the input distributions)
        spnk.layers.DenseSum(num_sums=num_input_distributions),
        # Randomly permute scopes and pad if necessary
        spnk.layers.PermuteAndPadScopesRandom(),
        # Products through reduction correspond to Einsum's notion of 
        # 'factorized' distributions
        spnk.layers.ReduceProduct(num_factors=int(np.ceil(num_vars / 2 ** depth))),
    ]
    for i in range(depth):
        # The alternating stacks of remaining product and sum layers
        sum_product_stack += [
            spnk.layers.DenseProduct(num_factors=2),
            spnk.layers.DenseSum(num_sums=num_sums if i < depth - 1 else 1),
        ]
    sum_product_stack.append(
        spnk.layers.RootSum(return_weighted_child_logits=False)
    )
    return spnk.models.SequentialSumProductNetwork(sum_product_stack)

train_ds = tf.data.Dataset.from_tensor_slices((train_x_orig,)).shuffle(10_000).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((valid_x_orig,)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_x_orig,)).batch(batch_size)

spn = build_ratspn(train_x_orig.shape[1], depth=depth)

spn.compile(
    loss=spnk.losses.NegativeLogLikelihood(),
    metrics=[spnk.metrics.LogLikelihood()],
    optimizer=spnk.optimizers.OnlineExpectationMaximization(learning_rate=0.05)
)

spn.summary()

start = time.time()
spn.fit(train_ds, validation_data=valid_ds, epochs=max_num_epochs)
print(f"Total time: {time.time() - start:.2f} seconds")

evaluation = spn.evaluate(test_ds)

## Analysis
We now see the following
- A test LLH of around -37.26 (not significantly different which makes sense)
- A total train time of **63.64** seconds
- Total number of trainable parameters 31,717,408 (same as above just as a sanity check)

In other words, `libspn-keras` is not just more flexible, but also more than twice as fast here!